# Generate Data

1. Copy stereo images into single folder
2. Rotate vertical images, so all images are aligned horizontally
3. From 1280 x 720px images, create 5 x 3 single images (l + r image) through windowing + slicing
4. Create anaglyph + inverse anaglyph from stereo images (with dame ID, but suffix)
5. Split data 70/30 into training and test data

## Imports

In [4]:
!pwd

/workspace/data/src/data_generation


In [6]:
# !pip install opencv-python

In [9]:
# !apt update && apt install -y libsm6 libxext6
# !apt-get install -y libxrender-dev

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]                
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [929 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]3m
Get:7 http://archive.ubuntu.com/ubuntu jammy/main amd64 Packages [1792 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy/restricted amd64 Packages [164 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB]
Get:10 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages [266 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports/universe amd64 Packages [32.2 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports/main amd64 Packages [81.0 kB]
Readin

In [ ]:
# !apt-get install libsm6 libxrender1 libfontconfig1

In [2]:
import numpy as np
import cv2 as cv2
import os
from sklearn.model_selection import train_test_split
import shutil
from tqdm import tqdm

In [2]:
source_folder = 'E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\holopix50k'
# source_folder = 'E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\test_data'
anaglyph_folder = 'E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\data\\anaglyph'
inverse_anaglyph_folder = 'E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\data\\inverse_anaglyph'
stereo_slice_folder = 'E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\data\\stereo'
stereo_folder = 'E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\data\\stereo_full'

In [6]:
# from PIL import Image
# 
# def get_unique_image_dimensions(folder_path):
#     image_sizes = set()  # Use a set to store unique image sizes
#     image_files = [filename for filename in os.listdir(folder_path) if filename.lower().endswith(('.jpg'))]
# 
#     # Create a tqdm progress bar
#     with tqdm(total=len(image_files), desc="Processing images") as pbar:
#         for filename in image_files:
#             image_path = os.path.join(folder_path, filename)
#             try:
#                 img = Image.open(image_path)
#                 width, height = img.size
#                 image_sizes.add((width, height))  # Add the size as a tuple to the set
#             except Exception as e:
#                 print(f"Error processing '{filename}': {e}")
#             pbar.update(1)  # Update the progress bar
# 
#     # Print the unique image sizes
#     for size in image_sizes:
#         print(f"Distinct image size: Width = {size[0]}, Height = {size[1]}")
# 
# # Example usage:
# get_unique_image_dimensions(source_folder)

Processing images: 100%|██████████| 49124/49124 [06:40<00:00, 122.52it/s]

Distinct image size: Width = 1280, Height = 360
Distinct image size: Width = 720, Height = 640
Distinct image size: Width = 1440, Height = 1280
Distinct image size: Width = 2560, Height = 720


## 2. Rotate images

In [12]:
def rotate_vertical_image(img):
    if img.shape[0] > img.shape[1]:
        img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    return img


## 3. Slice images

In [34]:
# print (320//256)

1


In [39]:
def create_image_chunks(image):
    img_height, img_width = image.shape[:2]  # Get image dimensions

    # Calculate chunk dimensions
    chunk_height = 256
    chunk_width = 256

    num_rows = (img_height // chunk_height) + 1
    num_cols = (img_width // chunk_width) + 1

    # Calculate overlap
    overlap_y = (img_height - num_rows * chunk_height) // num_rows
    overlap_x = (img_width - num_cols * chunk_width) // num_cols

    chunks = []
    for row in range(num_rows):
        for col in range(num_cols):
            y_start = row * (chunk_height - overlap_y)
            x_start = col * (chunk_width - overlap_x)
            y_end = min(y_start + chunk_height, img_height)
            x_end = min(x_start + chunk_width, img_width)

            # Only add chunks that meet the specified dimensions
            if y_end - y_start == chunk_height and x_end - x_start == chunk_width:
                chunks.append(image[y_start:y_end, x_start:x_end])

    return chunks

## 4. Create Anaglyph

In [25]:
def split_image(image):
    # Get the image shape
    height, width, _ = image.shape
    
    # Calculate the middle of the image
    middle = width // 2

    # Split the image into left and right halves
    left_img = image[:, :middle]
    right_img = image[:, middle:]

    return left_img, right_img

In [27]:
def merge_channels(img_left, img_right):
    # Split the images into B,G,R channels
    print(img_left.shape, img_right.shape)
    B_left, G_left, R_left = cv2.split(img_left)
    B_right, G_right, R_right = cv2.split(img_right)

    # Create a new image with the red channel from 'img_left' and the blue + green channel from 'img_right'
    merged_anaglyph = cv2.merge((B_right, G_right, R_left))
    merged_inverse_anaglyph = cv2.merge((B_left, G_left, R_right))

    return merged_anaglyph, merged_inverse_anaglyph

## Run for all

In [16]:
def process_all_images(source_folder, pbar):
     # Initialize a counter
    counter = 0
    
    # Walk through the source directory
    for file in os.listdir(source_folder):
        # Check if the file is a .jpg
        if file.endswith('.jpg'):
            # Create the full file paths
            src_file_path = os.path.join(source_folder, file)

            # Get image name + ext
            base_name, ext = os.path.splitext(file)
            
            # Read the image
            img = cv2.imread(src_file_path)

            # Rotate the image if vertical
            img = rotate_vertical_image(img)
            cv2.imwrite(os.path.join(stereo_folder, base_name + ext), img)
            
            # Split image
            img_left, img_right = split_image(img)
            
            # Create image slices
            img_chunks_left = create_image_chunks(img_left)
            img_chunks_right = create_image_chunks(img_right)
            
            # Create anaglyphs
            for i in range(len(img_chunks_left)):
                img_anaglyph, img_inverse_anaglyph = merge_channels(img_chunks_left[i], img_chunks_right[i])
                cv2.imwrite(os.path.join(stereo_slice_folder, base_name + f'_{i}_left' + ext), img_chunks_left[i])
                cv2.imwrite(os.path.join(stereo_slice_folder, base_name + f'_{i}_right' + ext), img_chunks_right[i])
                cv2.imwrite(os.path.join(anaglyph_folder, base_name + f'_{i}_anaglyph' + ext), img_anaglyph)
                cv2.imwrite(os.path.join(inverse_anaglyph_folder, base_name + f'_{i}_inverseanaglyph' + ext), img_inverse_anaglyph)

            # Increment the counter
            counter += 1
            # Update the progress bar
            pbar.update()

In [41]:
 # Get the total number of .jpg files in the source directory
total_files = len([file for file in os.listdir(source_folder) if file.endswith('.jpg')])



# Copy the images with a progress bar
with tqdm(total=total_files, desc="Processing images", unit="file") as pbar:
    process_all_images(source_folder, pbar)

Processing images: 100%|██████████| 1/1 [00:00<00:00, 43.48file/s]

360
1280
(256, 256, 3) (256, 256, 3)
(256, 256, 3) (256, 256, 3)


## Generate train and test data files

In [4]:
train_data_file_path = 'E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\train.txt'
test_data_file_path = 'E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\test.txt'

In [5]:
def split_images_to_train_test(image_directory, train_ratio=0.7):
    # Get a list of all image filenames in the directory
    image_files = [filename for filename in os.listdir(image_directory) if filename.lower().endswith('.jpg')]

    # Split the filenames into train and test sets
    train_files, test_files = train_test_split(image_files, test_size=1 - train_ratio, random_state=42)

    # Save the filenames to text files
    with open(train_data_file_path, 'w') as train_file:
        for filename in tqdm(train_files, desc="Saving training files"):
            train_file.write(filename + '\n')
    with open(test_data_file_path, 'w') as test_file:
        for filename in tqdm(test_files, desc="Saving test files"):
            test_file.write(filename + '\n')

    print(f"Saved {len(train_files)} training files and {len(test_files)} test files.")

In [6]:
split_images_to_train_test(source_folder)

Saving test files: 100%|██████████| 14738/14738 [00:00<00:00, 446570.67it/s]

Saved 34386 training files and 14738 test files.


## Remove suffixes

In [6]:
def remove_suffixes(source_folder, type_string, pbar):

    # Walk through the source directory
    for file in os.listdir(source_folder):
        if file.endswith('.jpg'):
            # Get image name + ext
            base_name, ext = os.path.splitext(file)
            suffix_length = len(type_string)
            base_name_trimmed = base_name[:-suffix_length]

            # rename
            os.rename(os.path.join(source_folder, base_name + ext), os.path.join(source_folder, base_name_trimmed + ext))

            # Update the progress bar
            pbar.update()

In [7]:
parent_folder  = 'E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\data'
anaglyph_folder = os.path.join(parent_folder, 'anaglyph')
inverse_anaglyph_folder = os.path.join(parent_folder, 'inverse_anaglyph')



 # Get the total number of .jpg files in the source directory
total_files = len([file for file in os.listdir(os.path.join(parent_folder, 'anaglyph')) if file.endswith('.jpg')])

# Copy the images with a progress bar
with tqdm(total=total_files * 2, desc="Processing images", unit="file") as pbar:
    remove_suffixes(anaglyph_folder, '_anaglyph', pbar)
    remove_suffixes(inverse_anaglyph_folder, '_inverseanaglyph', pbar)

Processing images: 100%|██████████| 4/4 [00:00<00:00, 1332.79file/s]


## Concat anaglyph and inverse anaglyph

In [1]:
from PIL import Image

im = Image.open('E:\\Programming\\HAW\\Grundprojekt\\anaglyph-computing\\data\\data\\anaglyph\\-LQzyFvfDr0ujcfCR5Vq_0.jpg')

In [9]:
from torch.utils.data import Dataset
from torchvision.transforms import transforms
to_tensor = transforms.ToTensor()

im = to_tensor(im).numpy()

In [10]:
im.shape[0]

3